In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np


In [2]:
data_gen = DataGenerator(random=False, base_dir = "data/btcusd/")
data_gen.rewind()
stateUtil = StateUtil(data_gen)

windows = 10

tec = TecAn(windows = windows)
data_count = 10

In [3]:
on_fly_data = []
tac_origin = []
data_gen.rewind()
for i in tqdm(range(data_count)):
    raw = data_gen.next()
    empty = []
    price = raw[stateUtil.PRICE_KEY]
    amount = raw[stateUtil.AMOUNT_KEY]
    tac_origin.append([price, amount]) 
    #print(len(tec.data))
    #print(tec.add_ta(empty, price, amount))
    on_fly_data.append(tec.add_ta(price, amount))


<ipython-input-3-ffd935035adc>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(data_count)):


  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
tec = TecAn(windows = windows)

def calcule_tecs(data):
    tecs = []
    print("Data {}".format(len(data)))
    df = pd.DataFrame(data, columns = ['Close', 'Volume'])
    close = df['Close']
    volume = df['Volume']
    for ta in tec.tas:
        value = ta(close, volume, 0.0, 0.0)
        tecs.append(value)
    return tecs

def add_tacs(index, result):
    list = []
    for tec in result:
        list.append(tec.iloc[index])
    return list

result = calcule_tecs(tac_origin)

branch = []

for i in tqdm(range(data_count)):
    tacs = add_tacs(i, result)
    branch.append(tacs)

print(len(branch))

Data 10


<ipython-input-4-e977c60a6938>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(data_count)):


  0%|          | 0/10 [00:00<?, ?it/s]

10


In [5]:
for data_index in range(data_count):
    f = on_fly_data[data_index]
    b = branch[data_index]
    same_values = f == b
    if (not same_values):
        for index in range(len(branch[0])):
            f_value = f[index]
            b_value = b[index]
            if (f_value != b_value):
                print("Data {} Col {}".format(data_index, index))
                print("{} {}".format(f_value, b_value))

In [6]:
index = -1
fly = on_fly_data[index]
bat = branch[index]
print(fly)
print(bat)
print(fly == bat)

[0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0, 0.0, 0.0, 0.0]
True


In [7]:
import pandas as pd
from ta.trend import *
from ta.momentum import *
from ta.volume import *
from ta.volatility import *
from ta import add_all_ta_features, add_trend_ta, add_volume_ta, add_volatility_ta, add_momentum_ta, add_others_ta
import matplotlib.pyplot as plt

In [8]:
df = pd.DataFrame(tac_origin, columns = ['Close', 'Volume'])
close = df['Close']
volume = df['Volume']

In [9]:
close.rolling(10, min_periods=2)

Rolling [window=10,min_periods=2,center=False,axis=0]

In [10]:
on_fly_data[1]

[0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [11]:
on_fly_data[-50]

IndexError: list index out of range

In [ ]:
on_fly_data

In [ ]:
from google.colab import drive 

In [ ]:
!mkdir drive

In [ ]:
import pandas as pd
import pandas_ta as ta
from pandas_datareader import DataReader
import datetime
import matplotlib.pyplot as plt

# Load data
symbol = 'AAPL'

num_of_years = 1
start_date = datetime.datetime.now() - datetime.timedelta(days=int(365.25 * num_of_years))
end_date = datetime.date.today()

df = DataReader(symbol, 'yahoo' ,start_date, end_date)

df.columns = map(str.lower, df.columns)

df.ta.strategy(name='all')

# Sanity check. Make sure all the columns are there
print(df.columns)
print(df.tail())
plt.plot(df['bbands'])